#### Packages

In [1]:
import keras
from keras import layers
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import os
from PIL import Image
%load_ext tensorboard

#### Dataset

Here we will be using the same dataset as the previous assignments in which the problem is to classify a set of images as cats or not retrieved from https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification?select=train. We removed the picture "dog_505.png" as it caused problems while preprocessing it. We will first process only 100 images per class (100 cats and 100 dogs for training and another 100 cats and 100 dogs for validation) to reduce their pixel resolution to the same as the other assignments (64 pixels x 64 pixels) and then represent them as arrays.

In [2]:
def preprocess_dataset(dataset):
    dataset_flatten = dataset.reshape(dataset.shape[0],-1).T
    return dataset_flatten/255 

def process_images(directory, target_size=(64, 64), image_range=range(0,100)):
    image_list = []

    filenames = sorted(os.listdir(directory))[min(image_range):max(image_range)+1]
    
    for filename in filenames:
        if filename.endswith(".jpg"):
            file_path = os.path.join(directory, filename)
            
            img = Image.open(file_path)
            img_resized = img.resize(target_size)
            img_array = np.array(img_resized)
            
            image_list.append(img_array)
    
    return np.array(image_list)

def join_cats_and_dogs(cat_images,dog_images):
    cat_set_X = preprocess_dataset(cat_images)
    dog_set_X = preprocess_dataset(dog_images)
    m_cat_set = cat_set_X.shape[1]
    cat_set_Y = np.ones((1, m_cat_set))
    m_dog_set = dog_set_X.shape[1]
    dog_set_Y = np.zeros((1, m_dog_set))

    set_X = np.concatenate((cat_set_X, dog_set_X), axis=1)
    set_Y = np.concatenate((cat_set_Y, dog_set_Y), axis=1)

    np.random.seed(1)
    shuffle_indices = np.random.permutation(set_X.shape[1])
    final_set_X = set_X[:, shuffle_indices]
    final_set_Y = set_Y[:, shuffle_indices]

    return final_set_X,final_set_Y


In [3]:
train_cat_images = process_images("datasets/train/cats")
train_dog_images = process_images("datasets/train/dogs")

test_cat_images = process_images("datasets/train/cats",image_range=(101,200))
test_dog_images = process_images("datasets/train/dogs",image_range=(101,200))

x_train, y_train = join_cats_and_dogs(train_cat_images,train_dog_images)
y_train = y_train.flatten()
x_train = x_train.T


x_test, y_test = join_cats_and_dogs(test_cat_images,test_dog_images)
y_test = y_test.flatten()
x_test = x_test.T


## Tuning Parameters

### Grid Search
We will tune 4 parameters: the number of units, the dropout frequency, the batch size and the optimizer used. 

In [4]:
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.05, 0.10, 0.15, 0.20]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([16, 32]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

In [5]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    # Add a Reshape layer to unflatten the input
    tf.keras.layers.Reshape((64, 64, 3), input_shape=(12288,)),
    
    # Block 1
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
    
    # Block 2
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2), strides=(2, 2)),
    
    # Flatten the output of the conv layers to feed into the dense layers
    tf.keras.layers.Flatten(),
    
    # Fully connected layers with dropout in between
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    
    # Output layer
    tf.keras.layers.Dense(2, activation='softmax'),  # Adjust the number of units for the number of classes
])

  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  callbacks = [
  keras.callbacks.TensorBoard(
  log_dir='logs/hparam_tuning',
  histogram_freq=1,
  embeddings_freq=1,
  )
  ]

  model.fit(x_train, y_train, epochs=1,callbacks=callbacks, batch_size=hparams[HP_BATCH_SIZE]) 
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [6]:
session_num = 0

for dropout_rate in HP_DROPOUT.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for batch_size in HP_BATCH_SIZE.domain.values:  # Add this loop
            hparams = {
                HP_DROPOUT: dropout_rate,
                HP_OPTIMIZER: optimizer,
                HP_BATCH_SIZE: batch_size,  # Include batch size
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1


--- Starting trial: run-0
{'dropout': 0.05, 'optimizer': 'adam', 'batch_size': 16}


: 

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11922263178460735227
 xla_global_id: -1]

In [ ]:
pip install "tensorflow-gpu<2.11"

   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/455.9 MB 1.4 MB/s eta 0:05:34
   ---------------------------------------- 0.1/455.9 MB 825.8 kB/s eta 0:09:12
   ---------------------------------------- 0.1/455.9 MB 1.0 MB/s eta 0:07:22
   ---------------------------------------- 0.2/455.9 MB 1.5 MB/s eta 0:05:02
   ---------------------------------------- 0.5/455.9 MB 2.2 MB/s eta 0:03:30
   ---------------------------------------- 0.6/455.9 MB 2.3 MB/s eta 0:03:17
   ---------------------------------------- 0.7/455.9 MB 2.4 MB/s eta 0:03:09
   ---------------------------------------- 0.9/455.9 MB 2.6 MB/s eta 0:02:52
   ---------------------------------------- 1.1/455.9 MB 2.9 MB/s eta 0:02:37
   ---------------------------------------- 1.2/455.9 MB 3.0 MB/s eta 0:02:32
   ---------------------------------------- 1.2/455.9 MB 3.0 MB/s eta 0:02:32
   ---------------------------------------- 1.2/455.9 MB 3.0 MB/s eta

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\Users\\13042\\anaconda3\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



In [5]:
tf.test.is_built_with_cuda()

False

In [6]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

tf.config.list_physical_devices('GPU')

from tensorflow.python.client import device_lib

device_lib.list_local_devices()

tf.test.is_built_with_cuda()

tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0


In [7]:
tf.config.list_physical_devices("GPU")

[]

In [ ]:
tf.config.list_physical_devices('GPU')

[]

In [ ]:
%tensorboard --logdir=logs/hparam_tuning

Reusing TensorBoard on port 6016 (pid 29612), started 0:00:14 ago. (Use '!kill 29612' to kill it.)